# AoC Day 12 part 1

- https://adventofcode.com/2024/day/12

- https://adventofcode.com/2024/day/12/input

In [1]:
import networkx as nx 
from pyvis.network import Network 
import copy


import sys
sys.path.insert(0, '../src/')
import day_12 as d12

## Example

In [2]:

garden_map_input = '''
RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE
'''

garden_map = [ list(a) for a in garden_map_input.split() ]

garden_map 

[['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'F', 'F'],
 ['R', 'R', 'R', 'R', 'I', 'I', 'C', 'C', 'C', 'F'],
 ['V', 'V', 'R', 'R', 'R', 'C', 'C', 'F', 'F', 'F'],
 ['V', 'V', 'R', 'C', 'C', 'C', 'J', 'F', 'F', 'F'],
 ['V', 'V', 'V', 'V', 'C', 'J', 'J', 'C', 'F', 'E'],
 ['V', 'V', 'I', 'V', 'C', 'C', 'J', 'J', 'E', 'E'],
 ['V', 'V', 'I', 'I', 'I', 'C', 'J', 'J', 'E', 'E'],
 ['M', 'I', 'I', 'I', 'I', 'I', 'J', 'J', 'E', 'E'],
 ['M', 'I', 'I', 'I', 'S', 'I', 'J', 'E', 'E', 'E'],
 ['M', 'M', 'M', 'I', 'S', 'S', 'J', 'E', 'E', 'E']]

In [3]:

def get_all_neighbours(x, y, map):
    n1_value ='.'
    if x>0 and y>0: 
        n1 = f'{x-1}_{y-1}'
        n1_value = map[y-1][x-1]
    
    n2_value ='.'
    if y>0: 
        n2 = f'{x}_{y-1}'
        n2_value = map[y-1][x]
        
    n3_value ='.'
    if x<len(map[0])-1 and y>0: 
        n3 = f'{x+1}_{y-1}'
        n3_value = map[y-1][x+1]
        
    n4_value ='.'
    if x< len(map[0])-1: 
        n4 = f'{x+1}_{y}'
        n4_value = map[y][x+1]
        
    n5_value ='.'
    if x< len(map[0])-1 and y< len(map)-1: 
        n5 = f'{x+1}_{y+1}'
        n5_value = map[y+1][x+1]
        
    n6_value ='.'
    if y< len(map)-1: 
        n6 = f'{x}_{y+1}'
        n6_value = map[y+1][x]
        
    n7_value ='.'
    if x>0 and y< len(map)-1: 
        n7 = f'{x-1}_{y+1}'
        n7_value = map[y+1][x-1]
        
    n8_value ='.'
    if x>0: 
        n8 = f'{x-1}_{y}'
        n8_value = map[y][x-1]
        
    return (n1_value, n2_value, n3_value, n4_value, n5_value, n6_value, n7_value, n8_value)

In [4]:
corners = dict() 

for y in range(len(garden_map)):
    for x in range(len(garden_map[0])):
        n_value = garden_map[y][x]
        n = f'{x}_{y}'
        
        n1_value, n2_value, n3_value, n4_value, n5_value, n6_value, n7_value, n8_value = get_all_neighbours(x, y, garden_map)
        
        if   n_value != n2_value and n_value != n4_value and n_value != n6_value and n_value != n8_value: corners[n] = 4     # no neighbours 1
        elif (n_value == n2_value and n_value != n4_value and n_value != n6_value and n_value != n8_value) or \
             (n_value != n2_value and n_value == n4_value and n_value != n6_value and n_value != n8_value) or \
             (n_value != n2_value and n_value != n4_value and n_value == n6_value and n_value != n8_value) or \
             (n_value != n2_value and n_value != n4_value and n_value != n6_value and n_value == n8_value): corners[n] = 2  # one neighbour 4
        elif n_value == n2_value and n_value == n4_value and n_value == n6_value and n_value == n8_value:  # four neighbours 1
            c = 4
            if n_value == n1_value: c = c - 1
            if n_value == n3_value: c = c - 1
            if n_value == n5_value: c = c - 1
            if n_value == n7_value: c = c - 1
            corners[n] = c     
        elif n_value == n2_value and n_value != n4_value and n_value == n6_value and n_value != n8_value: corners[n] = 0     # two oppisite neighbours 1
        elif n_value != n2_value and n_value == n4_value and n_value != n6_value and n_value == n8_value: corners[n] = 0     # two oppisite neighbours 1
        elif n_value == n2_value and n_value == n4_value and n_value != n6_value and n_value != n8_value:   # two corner neighbours 4
            if n_value == n3_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value != n2_value and n_value == n4_value and n_value == n6_value and n_value != n8_value:   # two corner neighbours 4
            if n_value == n5_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value != n2_value and n_value != n4_value and n_value == n6_value and n_value == n8_value:   # two corner neighbours 4
            if n_value == n7_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value == n2_value and n_value != n4_value and n_value != n6_value and n_value == n8_value:   # two corner neighbours 4
            if n_value == n1_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value == n2_value and n_value == n4_value and n_value == n6_value and n_value != n8_value:  # three neighbours 4
            c = 2
            if n_value == n3_value: c = c - 1
            if n_value == n5_value: c = c - 1
            corners[n] = c
        elif n_value != n2_value and n_value == n4_value and n_value == n6_value and n_value == n8_value:  
            c = 2
            if n_value == n5_value: c = c - 1
            if n_value == n7_value: c = c - 1
            corners[n] = c
        elif n_value == n2_value and n_value != n4_value and n_value == n6_value and n_value == n8_value:  
            c = 2
            if n_value == n7_value: c = c - 1
            if n_value == n1_value: c = c - 1
            corners[n] = c
        elif n_value == n2_value and n_value == n4_value and n_value != n6_value and n_value == n8_value:  
            c = 2
            if n_value == n1_value: c = c - 1
            if n_value == n3_value: c = c - 1
            corners[n] = c
        

        
        

In [5]:
def add_to_region(node, idx, regions_nodes, regions_mapping, regions):
    regions_nodes.append(node)
    regions_mapping[node] = idx
    regions[idx].append(node)
    
garden_map[1][4]

'I'

In [6]:



map_regions = dict() 
map_regions_nodes = list() 
map_regions_mapping = dict() 
region_idx = 0 


for y in range(len(garden_map)):
    for x in range(len(garden_map[0])):
        src = f'{x}_{y}'
        
for y in range(len(garden_map)):
    for x in range(len(garden_map[0])):
        
        src_value = garden_map[y][x]
        src = f'{x}_{y}'
    
        if src in map_regions_nodes:
            r_idx = map_regions_mapping[src] 
        else: 
            r_idx = region_idx
            map_regions[r_idx] = [src]
            map_regions_nodes.append(src) 
            map_regions_mapping[src] = r_idx
            region_idx = region_idx + 1
            
            fifo_queue = [src]
            
            while len(fifo_queue) > 0: 
                n1 = fifo_queue.pop(0)
                n1_x, n1_y = [int(v) for v in n1.split('_')]
                n1_value = garden_map[n1_y][n1_x]
                
                n2 = f'{n1_x}_{n1_y-1}'
                n2_value = '.'
                if n1_y > 0:
                    n2_value = garden_map[n1_y-1][n1_x]
                    if n2_value == n1_value : 
                        if n2 not in map_regions_nodes:
                            fifo_queue.append(n2)
                            add_to_region(n2, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                        
                n3 = f'{n1_x+1}_{n1_y}'
                n3_value = '.'
                if n1_x < len(garden_map[0])-1:
                    n3_value = garden_map[n1_y][n1_x+1]
                    if n3_value == n1_value : 
                        if n3 not in map_regions_nodes: 
                            fifo_queue.append(n3)
                            add_to_region(n3, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                        
                n4 = f'{n1_x}_{n1_y+1}'
                n4_value = '.'
                if n1_y < len(garden_map)-1:
                    n4_value = garden_map[n1_y+1][n1_x]
                    if n4_value == n1_value : 
                        if n4 not in map_regions_nodes: 
                            fifo_queue.append(n4)
                            add_to_region(n4, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                        
                n5 = f'{n1_x-1}_{n1_y}'
                n5_value = '.'
                if n1_x > 0: 
                    n5_value = garden_map[n1_y][n1_x-1]
                    if n5_value == n1_value : 
                        if n5 not in map_regions_nodes: 
                            fifo_queue.append(n5)
                            add_to_region(n5, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                

                
        
                        
                

In [7]:
# 1206

def get_fence_cost_sides(regions):
    cost = 0

            
    for key in regions: 
        region = regions[key]
        sides = sum( [ corners[r] for r in region] )
        cost = cost + ( len(region) * sides )
        x, y = [int(v) for v in list(region)[0].split('_')]
            
         
    
    return cost

get_fence_cost_sides(map_regions)

1206

## Input file

In [8]:
fp = '../data/day_12_1.txt'

with open(fp, 'r') as f_in:
   garden_map_input = f_in.readlines()

garden_map = [ list(a.strip()) for a in garden_map_input ]
#print( garden_map )
#garden_map


In [9]:
corners = dict() 

for y in range(len(garden_map)):
    for x in range(len(garden_map[0])):
        n_value = garden_map[y][x]
        n = f'{x}_{y}'
        
        n1_value, n2_value, n3_value, n4_value, n5_value, n6_value, n7_value, n8_value = get_all_neighbours(x, y, garden_map)
        
        if   n_value != n2_value and n_value != n4_value and n_value != n6_value and n_value != n8_value: corners[n] = 4     # no neighbours 1
        elif (n_value == n2_value and n_value != n4_value and n_value != n6_value and n_value != n8_value) or \
             (n_value != n2_value and n_value == n4_value and n_value != n6_value and n_value != n8_value) or \
             (n_value != n2_value and n_value != n4_value and n_value == n6_value and n_value != n8_value) or \
             (n_value != n2_value and n_value != n4_value and n_value != n6_value and n_value == n8_value): corners[n] = 2  # one neighbour 4
        elif n_value == n2_value and n_value == n4_value and n_value == n6_value and n_value == n8_value:  # four neighbours 1
            c = 4
            if n_value == n1_value: c = c - 1
            if n_value == n3_value: c = c - 1
            if n_value == n5_value: c = c - 1
            if n_value == n7_value: c = c - 1
            corners[n] = c     
        elif n_value == n2_value and n_value != n4_value and n_value == n6_value and n_value != n8_value: corners[n] = 0     # two oppisite neighbours 1
        elif n_value != n2_value and n_value == n4_value and n_value != n6_value and n_value == n8_value: corners[n] = 0     # two oppisite neighbours 1
        elif n_value == n2_value and n_value == n4_value and n_value != n6_value and n_value != n8_value:   # two corner neighbours 4
            if n_value == n3_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value != n2_value and n_value == n4_value and n_value == n6_value and n_value != n8_value:   # two corner neighbours 4
            if n_value == n5_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value != n2_value and n_value != n4_value and n_value == n6_value and n_value == n8_value:   # two corner neighbours 4
            if n_value == n7_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value == n2_value and n_value != n4_value and n_value != n6_value and n_value == n8_value:   # two corner neighbours 4
            if n_value == n1_value: corners[n] = 1
            else:                   corners[n] = 2
        elif n_value == n2_value and n_value == n4_value and n_value == n6_value and n_value != n8_value:  # three neighbours 4
            c = 2
            if n_value == n3_value: c = c - 1
            if n_value == n5_value: c = c - 1
            corners[n] = c
        elif n_value != n2_value and n_value == n4_value and n_value == n6_value and n_value == n8_value:  
            c = 2
            if n_value == n5_value: c = c - 1
            if n_value == n7_value: c = c - 1
            corners[n] = c
        elif n_value == n2_value and n_value != n4_value and n_value == n6_value and n_value == n8_value:  
            c = 2
            if n_value == n7_value: c = c - 1
            if n_value == n1_value: c = c - 1
            corners[n] = c
        elif n_value == n2_value and n_value == n4_value and n_value != n6_value and n_value == n8_value:  
            c = 2
            if n_value == n1_value: c = c - 1
            if n_value == n3_value: c = c - 1
            corners[n] = c
        

        
        

In [10]:


map_regions = dict() 
map_regions_nodes = list() 
map_regions_mapping = dict() 
region_idx = 0 


for y in range(len(garden_map)):
    for x in range(len(garden_map[0])):
        src = f'{x}_{y}'
        
for y in range(len(garden_map)):
    for x in range(len(garden_map[0])):
        
        src_value = garden_map[y][x]
        src = f'{x}_{y}'
    
        if src in map_regions_nodes:
            r_idx = map_regions_mapping[src] 
        else: 
            r_idx = region_idx
            map_regions[r_idx] = [src]
            map_regions_nodes.append(src) 
            map_regions_mapping[src] = r_idx
            region_idx = region_idx + 1
            
            fifo_queue = [src]
            
            while len(fifo_queue) > 0: 
                n1 = fifo_queue.pop(0)
                n1_x, n1_y = [int(v) for v in n1.split('_')]
                n1_value = garden_map[n1_y][n1_x]
                
                n2 = f'{n1_x}_{n1_y-1}'
                n2_value = '.'
                if n1_y > 0:
                    n2_value = garden_map[n1_y-1][n1_x]
                    if n2_value == n1_value : 
                        if n2 not in map_regions_nodes:
                            fifo_queue.append(n2)
                            add_to_region(n2, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                        
                n3 = f'{n1_x+1}_{n1_y}'
                n3_value = '.'
                if n1_x < len(garden_map[0])-1:
                    n3_value = garden_map[n1_y][n1_x+1]
                    if n3_value == n1_value : 
                        if n3 not in map_regions_nodes: 
                            fifo_queue.append(n3)
                            add_to_region(n3, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                        
                n4 = f'{n1_x}_{n1_y+1}'
                n4_value = '.'
                if n1_y < len(garden_map)-1:
                    n4_value = garden_map[n1_y+1][n1_x]
                    if n4_value == n1_value : 
                        if n4 not in map_regions_nodes: 
                            fifo_queue.append(n4)
                            add_to_region(n4, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                        
                n5 = f'{n1_x-1}_{n1_y}'
                n5_value = '.'
                if n1_x > 0: 
                    n5_value = garden_map[n1_y][n1_x-1]
                    if n5_value == n1_value : 
                        if n5 not in map_regions_nodes: 
                            fifo_queue.append(n5)
                            add_to_region(n5, r_idx, map_regions_nodes, map_regions_mapping, map_regions)
                

                
                     
                        
                

In [11]:
# 838988

get_fence_cost_sides(map_regions)

838988